In [2]:
!pip install agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.1/943.1 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 10.0 MB/s eta 0:00:00


In [3]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 30.9 MB/s eta 0:00:00


In [4]:
!pip install pillow


In [5]:
!pip install gradio


In [ ]:


import os
from PIL import Image as PILImage
from agno.agent import Agent
from agno.models.google import Gemini
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.media import Image as AgnoImage
import gradio as gr

# Load Google API Key
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Prompt for medical image analysis
query = """
You are a highly skilled medical imaging expert with extensive knowledge in radiology and diagnostic imaging. Analyze the patient's medical image and structure your response as follows:

### 1. Image Type & Region
- Specify imaging modality (X-ray/MRI/CT/Ultrasound/etc.)
- Identify the patient's anatomical region and positioning
- Comment on image quality and technical adequacy

### 2. Key Findings
- List primary observations systematically
- Note any abnormalities in the patient's imaging with precise descriptions
- Include measurements and densities where relevant
- Describe location, size, shape, and characteristics
- Rate severity: Normal/Mild/Moderate/Severe

### 3. Diagnostic Assessment
- Provide primary diagnosis with confidence level
- List differential diagnoses in order of likelihood
- Support each diagnosis with observed evidence from the patient's imaging
- Note any critical or urgent findings

### 4. Patient-Friendly Explanation
- Explain the findings in simple, clear language that the patient can understand
- Avoid medical jargon or provide clear definitions
- Include visual analogies if helpful
- Address common patient concerns related to these findings

### 5. Research Context
IMPORTANT: Use the DuckDuckGo search tool to:
- Find recent medical literature about similar cases
- Search for standard treatment protocols
- Provide a list of relevant medical links of them too
- Research any relevant technological advances
- Include 2-3 key references to support your analysis

Format your response using clear markdown headers and bullet points. Be concise yet thorough.
"""

# Image analysis function
def analyze_medical_image(api_key, image_np):
    if not api_key:
        return " Please provide a valid Google API key."

    try:
        # Initialize the agent
        agent = Agent(
            model=Gemini(id="gemini-2.0-flash", api_key=api_key),
            tools=[DuckDuckGoTools()],
            markdown=True
        )

        # Resize image
        img = PILImage.fromarray(image_np)
        img = img.resize((500, int(500 / img.width * img.height)))
        temp_path = "temp_image.png"
        img.save(temp_path)

        # Analyze with Gemini + DuckDuckGo
        agno_img = AgnoImage(filepath=temp_path)
        result = agent.run(query, images=[agno_img])

        return result.content

    except Exception as e:
        return f" Analysis error: {e}"

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("##  Medical Imaging Diagnosis Agent")
    gr.Markdown("Upload a medical image to receive an AI-generated analysis report.\n\n **Disclaimer:** Educational use only. Not a clinical diagnosis tool.")

    api_input = gr.Textbox(label=" Google API Key", type="password", placeholder="Paste your API key")
    image_input = gr.Image(label="Upload Medical Image", type="numpy")
    analyze_button = gr.Button(" Analyze")
    output_md = gr.Markdown()

    analyze_button.click(analyze_medical_image, inputs=[api_input, image_input], outputs=output_md)

demo.launch()

